In [106]:
# Initial setup
# Set up a dictionary of signs
signs = dict()
signs['union'] = 'v'
signs['intersection'] = '^'
signs['implication'] = '→'
signs['negation'] = '~'

# Set up list of atomic statements
atomic = []
atomic.append('A')
atomic.append('B')
atomic.append('C')
atomic.append('D')
atomic.append('E')

In [ ]:
# To-do: Komla
# Function to transform parenthesis form (a->b)->(~b->~a) to non-parenthesis form ->->ab->~b~a

In [107]:
# Function to split text into Alpha, Beta, S1 and S2
def alpha_beta_split(text):

    # Support for switching between alpha to beta
    alpha_count = 0
    label = 'S1'

    # Dictionary of alpha/beta/S strings
    ab_dict = dict()
    ab_dict['alpha'] = ''
    ab_dict['beta'] = ''
    ab_dict['S1'] = ''
    ab_dict['S2'] = ''

    for i in range(len(text)):  # Loop over every char
        # Switch labels to beta or S2 if conditions are satisfied
        if alpha_count >= 1 and label == 'alpha':
            label = 'beta'
        elif text[i] == ',' and label == 'beta':
            label = 'S2'

        # If current char is a sign
        if text[i] in signs.values():
            if text[i] == signs['negation']:    # Ignore alpha_count for negation
                ab_dict[label] += text[i]       # Add negation to alpha/beta/S
            else:
                if label == 'S1':               # Marks start of alpha
                    label = 'alpha'
                else:
                    ab_dict[label] += text[i]       # Add sign to alpha/beta/S
                    alpha_count -= 1                # Reduce alpha_count to add 1 more atomic statement to alpha

        # If current char is an Atomic statement
        elif text[i] in atomic:
            if label != 'alpha':        # Ignore alpha_count for other labels
                ab_dict[label] += text[i]
            else:
                ab_dict[label] += text[i]
                alpha_count += 1

        # If current char is indecomposable sequence
        else:
            ab_dict[label] += text[i]

    return ab_dict

In [108]:
def union(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [109]:
# Todo: Use other funcs to define intersection, negative union, and negation

# def neg_union(text):

In [110]:
# def intersection(text):

In [111]:
# def double_negation(text):

In [ ]:
# def neg_implication(text):

In [112]:
def neg_intersection(text):
    text = text.replace(signs['negation'], '', 1)
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + signs['negation'] + ab_dict['beta'] + ab_dict['S2']

In [113]:
def implication(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [ ]:
# To-do:
# Helper function to check if a leaf is fundamental

In [ ]:
# To-do:
# Function to call the respective schema function based on current operation and handle output
# Feature: Perform depth-first search, output leaf values and if equation is Tautology

In [ ]:
# To-do:
# Implement Console/UI for Input/Output + write documentation

In [114]:
# Test input
input = '→→A→BC→→AB→AC'
output = implication(input)
print(output)

~→A→BC,→→AB→AC


In [115]:
# Test input
input = '→~B~A,~→AB'
output = implication(input)
print(output)

~~B,~A,~→AB


In [116]:
# Test input
input = 'P,vAB,Q'       # P = S1, Q = S2
output = union(input)
print(output)

P,A,B,Q


In [117]:
# Test input
input = 'vAB,P,Q'      # P, Q = S2
output = union(input)
print(output)

A,B,P,Q


In [118]:
# Test input
input = 'P,~^AB,Q'          # P = S1, Q = S2
output = neg_intersection(input)
print(output)

P,~A,~B,Q


In [119]:
# Test input for schema [S1,~(AvB),S2]/[S1,~A,S2;S1,~B,S2]
input1 = 'P,~vAB,Q'
# output1 = neg_union(input1)
input2 = 'P,~A,Q'
# output2 = negation(input2)
input3 = 'P,~B,Q'
# output3 = negation(input3)
# input = '[input1]/[input2;input3]'
# output =
# print(output)

In [120]:
# Test input for schema [S1,~(A->B),S2]/[S1,A,S2;S1,~B,S2]
input1 = 'P,~→AB,Q'
output1 = implication(input1)
input2 = 'P,A,Q'
input3 = 'P,~B,Q'
# input = '[input1]/[input2;input3]'
# output = 
# print(output)